# Adsorption Energies

Adsorption energies are the core of computational catalysis and surface science because they provide fundamental information about how a molecular intermediate interacts with a catalyst surface. Computational approaches are particularly valuable for adsorption energies because they are exceedingly difficult to measure experimentally. However, calculating adsorption energies also requires some effort. This post is meant to cover the basics of how to calculate an adsorption energy using DFT.

The adsorption energy is defined as the energy difference between the combined system and the separate systems:

$E_{ads} = E_{surf+ads} - E_{surf} - E_{gas}$

where $E_{surf+ads}$ is the combined system, $E_{surf}$ is the energy of the surface, and $E_{gas}$ is the energy of the molecule in the gas-phase. 

We'll start by just importing the library we use for manipulating atoms, the atomistic simulation environment (ASE), and calculating the energy of a gas molecule $E_{gas}$

In [1]:
from ase.build import molecule

gas = molecule('CO')

This code just makes a CO molecule, lets visualize it next.

In [2]:
from ase.visualize import view

view(gas, viewer = 'ngl')

To calculate the energy of this molecule, we're going to use the EMT calculator. EMT is a classical calculator (as opposed to quantum mechanical.) This code will give you the "potential energy" of the CO molecule

In [3]:
from ase.calculators.emt import EMT

gas.set_calculator(EMT())
gas.get_potential_energy()

0.7862094936389532

Next we need to optimize the sctructure, because chances are it's not in its lowest energy configuration

In [4]:
from ase.optimize import QuasiNewton

dyn = QuasiNewton(gas)
dyn.run(fmax=0.05)


E_gas = gas.get_potential_energy()

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 10:52:42        0.786209*       2.3049
BFGSLineSearch:    1[  2] 10:52:42        0.672187*       0.3302
BFGSLineSearch:    2[  3] 10:52:42        0.670368*       0.0714
BFGSLineSearch:    3[  4] 10:52:42        0.670282*       0.0016


In [5]:
from ase.build import fcc111, add_adsorbate

slab = fcc111('Pt', size = (2,2,4), vacuum = 10)
view(slab, viewer = 'ngl')

slab.set_calculator(EMT())
dyn = QuasiNewton(slab)
dyn.run(fmax=0.05)

E_slab = slab.get_potential_energy()

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 10:52:43        2.576181*       0.3811
BFGSLineSearch:    1[  1] 10:52:43        2.506582*       0.2684
BFGSLineSearch:    2[  2] 10:52:43        2.485247*       0.0649
BFGSLineSearch:    3[  3] 10:52:43        2.480666*       0.0440


In [6]:
add_adsorbate(slab, gas, 2.3,'hcp')
view(slab, viewer = 'ngl')

slab.set_calculator(EMT())
dyn = QuasiNewton(slab)
dyn.run(fmax=0.05)

E_slab_ads = slab.get_potential_energy()

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 10:52:44        3.346721*       3.8864
BFGSLineSearch:    1[  1] 10:52:44        2.976056*       2.4870
BFGSLineSearch:    2[  2] 10:52:44        2.810946*       1.0161
BFGSLineSearch:    3[  3] 10:52:44        2.692702*       0.5799
BFGSLineSearch:    4[  4] 10:52:44        2.648115*       0.2611
BFGSLineSearch:    5[  5] 10:52:44        2.625641*       0.1785
BFGSLineSearch:    6[  6] 10:52:44        2.618584*       0.0506
BFGSLineSearch:    7[  7] 10:52:44        2.617786*       0.0117


In [7]:
view(slab, viewer = 'ngl')

In [8]:
E_ads = E_slab_ads - E_slab - E_gas
print(E_ads)

-0.5331615226207451
